# Tutorial for HP1alpha dimer slab simulation.

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit
import mdtraj

sys.path.append('../../')
from openabc.forcefields.parsers import MOFFParser
from openabc.forcefields import MOFFMRGModel
import openabc.utils.helper_functions as helper_functions
from openabc.utils.insert import insert_molecules
from openabc.utils.CA2AA import multiple_chains_CA2AA

# set simulation platform
platform_name = 'CPU'

## 1. Build single HP1alpha dimer

Following steps in single_hp1alpha_dimer.ipynb, we build a single hp1alpha dimer.

In [2]:
hp1alpha_dimer_parser = MOFFParser.from_atomistic_pdb('input-pdb/hp1a.pdb', 'hp1alpha_dimer_CA.pdb')

old_native_pairs = hp1alpha_dimer_parser.native_pairs.copy()
new_native_pairs = pd.DataFrame(columns=old_native_pairs.columns)
cd1 = np.arange(16, 72)
csd1 = np.arange(114, 176)
n_atoms_per_hp1alpha_dimer = len(hp1alpha_dimer_parser.atoms.index)
print(f'There are {n_atoms_per_hp1alpha_dimer} CA atoms in each HP1alpha dimer.')
cd2 = cd1 + int(n_atoms_per_hp1alpha_dimer/2)
csd2 = csd1 + int(n_atoms_per_hp1alpha_dimer/2)
for i, row in old_native_pairs.iterrows():
    a1, a2 = int(row['a1']), int(row['a2'])
    if a1 > a2:
        a1, a2 = a2, a1
    flag1 = ((a1 in cd1) and (a2 in cd1)) or ((a1 in csd1) and (a2 in csd1))
    flag2 = ((a1 in cd2) and (a2 in cd2)) or ((a1 in csd2) and (a2 in csd2))
    flag3 = ((a1 in csd1) and (a2 in csd2))
    if flag1 or flag2 or flag3:
        new_native_pairs.loc[len(new_native_pairs.index)] = row
hp1alpha_dimer_parser.native_pairs = new_native_pairs
hp1alpha_dimer_parser.parse_exclusions() # update exclusions based on the new native pairs

Parse molecule with default settings.
Get native pairs with shadow algorithm.
There are 382 CA atoms in each HP1alpha dimer.


## 2. Slab simulation of 20 HP1alpha dimers.

Here we show the example of running 20 HP1alpha dimers with MOFF. This is a rather small system only as an example, and to study phase behavior you should add more molecules. Following these steps, you can easily extend to slab simulations with more molecules.

To begin with building the initial configuration that 20 dimers are well distributed in the box, we use `insert_molecules` to insert molecules into a 50x50x50 nm^3 box. 

In [3]:
n_mol = 20
insert_molecules('hp1alpha_dimer_CA.pdb', 'start.pdb', n_mol, box=[50, 50, 50])

Check contact with FastNS method. 
Successfully inserted 20 molecules.


We begin the simulation with NPT compression. For simplicity, in the jupyter notebook we only run 200 steps. 

To perform NPT simulation, we use `MonteCarloBarostat`. Note the NPT compression is performed under 1 bar and 150 K condition. However, the parameters for electrostatic interactions are defined based on 300 K. This is because finally we will perform the slab simulation at 300 K, so we keep using the Hamiltonian corresponding to 300 K. 

In [4]:
multi_dimers = MOFFMRGModel()
for i in range(n_mol):
    # append multiple hp1alpha dimer parser instances
    multi_dimers.append_mol(hp1alpha_dimer_parser)
multi_dimers.native_pairs.loc[:, 'epsilon'] = 6.0

box_a, box_b, box_c = 50, 50, 50
top = app.PDBFile('start.pdb').getTopology()
multi_dimers.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
init_coord = app.PDBFile('start.pdb').getPositions()
salt_conc = 82*unit.millimolar

multi_dimers.add_protein_bonds(force_group=1)
multi_dimers.add_protein_angles(force_group=2)
multi_dimers.add_protein_dihedrals(force_group=3)
multi_dimers.add_native_pairs(force_group=4)
multi_dimers.add_contacts(force_group=5)
multi_dimers.add_elec_switch(salt_conc, temperature=300*unit.kelvin, force_group=6)

# follow the example provided by OpenMM user guide, use LangevinMiddleIntegrator and MonteCarloBarostat to perform NPT simulation
pressure = 1*unit.bar
temperature = 150*unit.kelvin
multi_dimers.system.addForce(mm.MonteCarloBarostat(pressure, temperature))
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
multi_dimers.set_simulation(integrator, platform_name, init_coord=init_coord)
multi_dimers.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_NPT.dcd'
multi_dimers.add_reporters(output_interval, output_dcd)
multi_dimers.simulation.context.setVelocitiesToTemperature(temperature)
multi_dimers.simulation.step(200)
    

Add protein bonds.
Add protein angles.
Add protein dihedrals.
Add native pairs.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,-58589.888342060425,11821.981471836212,-46767.906870224215,124.0875815236911,0
200,2.0000000000000013,-56669.70066997619,13185.64860701582,-43484.05206296037,138.40110055693444,7.84


After we get the compressed configuration, we can perform slab simulation by putting it into an elongated box and performing NVT simulation. 

The NPT compressed configuration is performed with script "compress.py", and "NPT-output-files/NPT_compress.dcd" is the trajectory. We start from the final snapshot of NPT-output-files/NPT_compress.dcd. We place the compressed condensate into a box of size 15x15x200 nm^3. If you have more molecules then you should also enlarge your slab simulation box adaptively. 

In [5]:
# set slab simulation box size
box_a = 15
box_b = 15
box_c = 200

# load trajectory and get the compressed configuration
# for easier visualization, we move the geometric center of all the atoms to the box center
npt_traj = mdtraj.load_dcd('NPT-output-files/NPT_compress.dcd', top='start.pdb')
init_coord = npt_traj.xyz[-1]
init_coord -= np.mean(init_coord, axis=0)
init_coord += 0.5*np.array([box_a, box_b, box_c])

# we have to rebuild the system as this time there is no MonteCarloBarostat in it
multi_dimers = MOFFMRGModel()

for i in range(n_mol):
    # append multiple hp1alpha dimer parser instances
    multi_dimers.append_mol(hp1alpha_dimer_parser)

multi_dimers.native_pairs.loc[:, 'epsilon'] = 6.0

top = app.PDBFile('start.pdb').getTopology()
multi_dimers.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
multi_dimers.add_protein_bonds(force_group=1)
multi_dimers.add_protein_angles(force_group=2)
multi_dimers.add_protein_dihedrals(force_group=3)
multi_dimers.add_native_pairs(force_group=4)
multi_dimers.add_contacts(force_group=5)
multi_dimers.add_elec_switch(salt_conc, temperature, force_group=6)

# use Nose-Hoover integrator to accelerate the dynamics
collision = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.NoseHooverIntegrator(temperature, collision, timestep)
multi_dimers.set_simulation(integrator, platform_name, init_coord=init_coord)
multi_dimers.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_slab.dcd'
multi_dimers.add_reporters(output_interval, output_dcd)
multi_dimers.simulation.context.setVelocitiesToTemperature(temperature)
multi_dimers.simulation.step(200)

Add protein bonds.
Add protein angles.
Add protein dihedrals.
Add native pairs.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,-56867.97647594667,15445.350218045944,-41422.62625790073,162.1197054748936,0
200,2.0000000000000013,-55564.991061389934,17166.22405523305,-38398.76700615688,180.18258884792726,3.5


## 3. Convert to all atom configurations

So we can also convert the C-alpha CG configuration to all-atom representations. The input CA pdb is slab_NVT_relaxed.pdb, and the output AA pdb is slab_NVT_relaxed_AA.pdb. 

We use REMO to convert C-alpha model to all-atom model. To use REMO with our functions to conveniently do the conversion, please download [REMO](https://zhanggroup.org/REMO/REMO.v3.tar.bz2). By default we put REMO directory in openabc/utils (note REMO is not added in GitHub repository) and we do not need to specify the path whey using related functions. If you put it in other paths, please specify REMO path in `multiple_chains_CA2AA` with parameter `REMO_path`. 

In [6]:
run_convert = False # set to True if you want to run this part, which is time consuming

if run_convert:
    # start from the final snapshot of the previous short slab NVT trajectory
    state = multi_dimers.simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
    positions = np.array(state.getPositions().value_in_unit(unit.nanometer))

    # write CA pdb with the target positions
    df_atoms = helper_functions.parse_pdb('start.pdb')
    df_atoms.loc[:, ['x', 'y', 'z']] = positions*10 # convert nm to A
    helper_functions.write_pdb(df_atoms, 'slab_NVT_relaxed.pdb')

    # convert CA pdb to AA pdb
    # note each molecule has 2 monomers, and each monomer has 191 residues
    # thus there are 2*n_mol chains, and each chain has 191 residues
    multiple_chains_CA2AA('slab_NVT_relaxed.pdb', [2*n_mol], [191])


After converting to all-atom configurations, you can run all-atom simulations with implicit or explicit solvent models, and with your favourite simulation software. For example, GROMACS is a popular program for running all-atom simulations. OpenMM can also handle all-atom simulations. 